### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [65]:
# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# # Create data subset
# user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
# ratings_mat = np.matrix(user_movie_subset)
# print(ratings_mat)

In [2]:
user_movie_subset = pd.read_csv('user_movie_subset.csv', index_col=0)
user_movie_subset

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD. 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [48]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] 
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) 
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize set at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i,j] > 0:
                    
                    # compute the error 
                    diff = ratings_mat[i,j] - np.dot(user_mat[i,:], movie_mat[:,j])
                    
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += np.power(diff, 2)
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i,k] += learning_rate * (2*diff*movie_mat[k,j])
                        movie_mat[k,j] += learning_rate * (2*diff*user_mat[i,k])

        # print results for iteration
        print(f'{iteration+1}\t\t{sse_accum/num_ratings}')
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [51]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimization Statistics
Iterations | Mean Squared Error 
1		46.20574670181012
2		16.861158395210005
3		4.419656437714018
4		2.900348938387209
5		2.785668567128718
6		2.758126422351558
7		2.739753673426836
8		2.721523487939952
9		2.701796678127556
10		2.6801752880023058


In [52]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.88785527   8.59887975  10.87488376  10.20073978]
 [  8.4399865    7.45467334   9.28400076   8.72161563]
 [  7.93484959   6.97268933   8.09480256   7.83004541]
 [  9.39682333   8.01036415  10.20236153   9.70818853]
 [  8.26093168   7.23636882   9.23094213   8.59286313]
 [  6.6822743    5.7368538    7.35631114   6.9714263 ]
 [  9.38897176   8.22214254   9.44507464   9.21438421]
 [  8.3601922    7.19851152   8.46406365   8.22699919]
 [  8.4085194    7.28581401   8.96143819   8.58313166]
 [  7.84921265   6.91568093   7.95649905   7.56897635]
 [  8.81209153   7.58368856   9.36839277   8.98258827]
 [  9.73894681   8.62095024   9.96027997   9.47610241]
 [  9.26173791   8.29716226   9.76660133   9.13309265]
 [  6.72025919   5.90547999   6.89031565   6.57829265]
 [  9.88428075   8.74176345  10.02760327   9.72314593]
 [  9.83249896   8.61453549  10.08500243   9.76626007]
 [  8.42467466   7.28253494   8.53476945   8.33938976]
 [  7.27334256   6.31700354   7.28118553   7.05683282]
 [  8.2748

The predicted ratings from the dot product are already starting to look a lot like the original data values even after only 10 iterations.  You can see some extreme low values that are not captured well yet.  The 5 in the second to last row in the first column is predicted as an 8, and the 4 in the second row and second column is predicted to be a 7.  Clearly the model is not done learning, but things are looking good.

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [55]:
user_mat, movie_mat = FunkSVD(ratings_mat, learning_rate=0.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 
1		43.55528149872617
2		15.355277286565638
3		4.096206214169096
4		2.7054482612934674
5		2.5830106834783817
6		2.543856161887332
7		2.510702730000472
8		2.475303805046307
9		2.4363647726846587
10		2.3937312490144222
11		2.3474245119988746
12		2.2975024570925346
13		2.244059485722191
14		2.1872357430894223
15		2.1272168379080325
16		2.0642251856065363
17		1.9985071193671229
18		1.930320198174261
19		1.8599248386502656
20		1.7875834650874296
21		1.713568649010058
22		1.6381793324464418
23		1.5617616622562878
24		1.484728880744123
25		1.4075738043784367
26		1.3308681433524607
27		1.2552453235666474
28		1.1813671146093452
29		1.1098783353070114
30		1.0413570755836208
31		0.9762692603537142
32		0.9149355299142814
33		0.8575155908249885
34		0.8040113108002496
35		0.7542860797961649
36		0.7080953443041309
37		0.6651222294393604
38		0.6250126863101395
39		0.5874061147235791
40		0.5519592590785435
41		0.5183628241583144
42		0.486351415134

In [57]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.00000329  10.00000114  10.00000089   9.99999688]
 [ 10.00000185   4.00000061   9.00000046   9.99999823]
 [  7.99999989   9.00000001   9.99999999   5.        ]
 [  8.99999923   7.9999998    9.99999982  10.00000059]
 [  9.99999341   4.99999799   8.99999841   9.00000594]
 [  6.00000672   4.00000219  10.00000172   5.99999378]
 [  9.00002044   8.00000639  10.00000498   8.99998118]
 [ 10.00000003   4.99999982   8.99999977   7.99999985]
 [  6.99999762   7.99999906   9.99999917   8.00000201]
 [  8.99999832   4.99999933   8.99999941   7.00000143]
 [  8.99999851   7.9999994    9.99999946   8.00000123]
 [  9.00000413  10.00000117  10.00000085   8.99999606]
 [ 10.0000003    8.99999993   9.99999987   7.99999957]
 [  4.9999986    7.99999946   4.99999952   8.00000115]
 [ 10.00000803   8.00000236  10.00000177   9.9999925 ]
 [  8.99999676   8.99999876   9.99999892  10.00000277]
 [  8.99999898   7.99999954   7.99999957   8.00000079]
 [  9.99999938   7.99999975   0.99999977  10.00000047]
 [  4.9999

In this case, we were able to completely reconstruct the item-movie matrix to obtain an essentially 0 mean squared error, near 119 interation.

If we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using FunkSVD function. 

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [60]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan

In [61]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, learning_rate=0.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 
1		39.99492715105232
2		13.102248795900666
3		3.9818264148901963
4		2.8641560218516267
5		2.7005017970454896
6		2.636308484575618
7		2.5891347256755566
8		2.5425489275337676
9		2.4921677340144517
10		2.436513084232694
11		2.3749096409291828
12		2.3070415919574017
13		2.2328989448466867
14		2.1528073234543466
15		2.0674611494144677
16		1.9779308267510358
17		1.8856279939745162
18		1.7922216537777984
19		1.6995089642493024
20		1.6092570021344363
21		1.5230420762324457
22		1.4421167513689512
23		1.367329199932595
24		1.29910618329127
25		1.2374947726951917
26		1.1822448774588608
27		1.1329088814313228
28		1.0889367110462023
29		1.0497518684119669
30		1.0148025925508912
31		0.9835893474076677
32		0.9556739162933597
33		0.9306766308731972
34		0.9082675868086065
35		0.8881561259518114
36		0.8700811692307681
37		0.8538035888146471
38		0.8391008545774126
39		0.8257636510487034
40		0.8135939369782299
41		0.8024038984301834
42		0.792015333

In [62]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.766603170163838:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [66]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, learning_rate=0.005, iters=20)

Optimization Statistics
Iterations | Mean Squared Error 
1		23.35017282669142
2		10.811599995209313
3		7.400599399853698
4		5.706975257407862
5		4.6420673675673765
6		3.8956979079061305
7		3.339465360441204
8		2.90865757838278
9		2.5659530200785365
10		2.287827623341719
11		2.058560236549338
12		1.86720447628408
13		1.7058891893859558
14		1.568793933946609
15		1.4515022684194672
16		1.3505812037518397
17		1.263303625227244
18		1.1874638758898763
19		1.1212537700421017
20		1.063177000462017


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [74]:
# Replace each of the comments below with the correct values
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't actually have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [75]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
